# "Web Scraping Assignment-3"

In [1]:
# Importing Libraries
import selenium
import pandas as pd
import time
from bs4 import BeautifulSoup

# Importing selenium webdriver 
from selenium import webdriver
import warnings
warnings.filterwarnings('ignore')

# Importing required Exceptions which needs to handled
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
from selenium.webdriver.common.by import By
#Importing requests
import requests

# importing regex
import re

# Q1. Write a python program which searches all the product under a particular product from www.amazon.in. The product to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for guitars.

In [2]:
# connect to webdriver
driver=webdriver.Chrome(r"chromedriver.exe") 
time.sleep(2)

# to get url
url = "https://www.amazon.in/"
driver.get(url)

time.sleep(2)
# input the keywords that user wants to search
user_input = input('Enter the product you want to search : ')

Enter the product you want to search : Guitar


In [3]:
# to Locate searc_bar by id
search_bar = driver.find_element(By.ID,"twotabsearchtextbox") 
# to clear the search_bar
search_bar.clear() 
# to send user input to search bar
search_bar.send_keys(user_input)
 # to Locate search_button by xpath
search_button = driver.find_element(By.XPATH,'//div[@class="nav-search-submit nav-sprite"]/span/input')      
search_button.click()        

# Q2. In the above question, now scrape the following details of each product listed in first 3 pages of your search results and save it in a data frame and csv. In case if any product has less than 3 pages in search results then scrape all the products available under that product name. Details to be scraped are: "Brand Name", "Name of the Product", "Price", "Return/Exchange", "Expected Delivery", "Availability" and “Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“.

In [4]:
#collecting all the Product URLS
urls = []
for i in range(0,3):
    Page_urls=driver.find_elements(By.XPATH,'//a[@class="a-link-normal s-no-outline"]')#collecting urls of all the guitar
    for i in Page_urls:
        urls.append(i.get_attribute('href'))
        
    #next button 
    nxt_btn=driver.find_element(By.XPATH,'//a[@class="s-pagination-item s-pagination-next s-pagination-button s-pagination-separator"]') 
    url=nxt_btn.get_attribute('href')
    driver.get(url)
    time.sleep(2)

In [5]:
#Making empty lists and scraping the required spots
Product_name = []
Brand_name= []
Price = []
Return = []
Expected_Delivery = []
Availability = []

#Start with for loop
for i in urls:
    driver.get(i)
    time.sleep(2)
    
    
    #Scraping product name
    try:
        prod=driver.find_element(By.XPATH,'//span[@class="a-size-base-plus a-color-base a-text-normal"]')
        Product_name.append(prod.text)
    except NoSuchElementException as e:
        Product_name.append("-")
        
        
    #Scraping brand name
    try:
        brand=driver.find_element(By.XPATH,'//div[@id="bylineInfo_feature_div"]/div/a')
        Brand_name.append(brand.text)
    except NoSuchElementException as e:
        Brand_name.append("-")
        
        
    #Scraping Price
    try:
        pri=driver.find_element(By.XPATH,'//span[@class="a-price-whole"]')  
        Price.append(pri.text)
    except NoSuchElementException as e:
        Price.append("-")
        
        
    #Scraping Return/Exchange
    try:
        ret=driver.find_element(By.XPATH,'//div[@data-name="RETURNS_POLICY"]/span/div[2]/a')
        Return.append(ret.text)
    except NoSuchElementException as e:
        Return.append("-")
        
     
    #Scraping  Expected_Delivery
    try:
        delivary=driver.find_element(By.XPATH,'//span[@class="a-color-base a-text-bold"]')
        Expected_Delivery.append(delivary.text)
    except NoSuchElementException as e:
        Expected_Delivery.append("-")
        
        
    #Scraping Availability
    try:
        avai=driver.find_element(By.XPATH,'//div[@id="availability"]')
        Availability.append(avai.text)
    except NoSuchElementException as e:
        Availability.append("-")

        
# To make dataframe
Guitar=pd.DataFrame({})
Guitar['Name'] = Product_name
Guitar['Brand'] = Brand_name
Guitar['Price'] = Price
Guitar['Return'] = Return
Guitar['Expected_Delivery'] = Expected_Delivery
Guitar['Availability'] = Availability
Guitar['Urls'] = urls

In [6]:
#Print dataframe
Guitar

,Name,Brand,Price,Return,Expected_Delivery,Availability,Urls
0,-,Visit the Kadence Store,"9,999",7 Days Replacement,-,,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
1,-,Brand: FESTRA,"2,695",7 Days Replacement,-,In stock.,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
2,-,Visit the VAULT Store,"7,290",7 Days Replacement,-,In stock.,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
3,-,Brand: LUCHILA,965,7 Days Replacement,-,In stock.,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
4,-,Visit the Juârez Store,"2,399",7 Days Replacement,-,In stock.,https://www.amazon.in/Juarez-Acoustic-Cutaway-...
...,...,...,...,...,...,...,...
160,-,Brand: Pluto,"7,100",7 Days Replacement,-,In stock.,https://www.amazon.in/Pluto-HW41C201-HW41C-201...
161,-,Brand: Belear,"4,399",7 Days Replacement,-,In stock.,https://www.amazon.in/Belear-M-40-Orange-Class...
162,-,Brand: Epiphone,"15,500",7 Days Replacement,-,Only 1 left in stock.,https://www.amazon.in/Epiphone-DR-100-Acoustic...
163,-,Visit the JUAREZ Store,"5,528",7 Days Replacement,-,In stock.,https://www.amazon.in/JUAREZ-Arp%C3%A9ggio-Sem...


In [7]:
driver.close

<bound method WebDriver.close of <selenium.webdriver.chrome.webdriver.WebDriver (session="60bc070964fb4dc71664a2842449eeb8")>>

# Q3.Write a python program to access the search bar and search button on images.google.com and scrape 10 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’, ‘Guitar’, ‘Cakes’.

In [8]:
import selenium
import pandas as pd
from selenium import webdriver
import warnings
warnings.filterwarnings('ignore')
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
from selenium.webdriver.common.by import By
import time
import requests

In [9]:
driver = webdriver.Chrome(r"chromedriver.exe")
url='https://images.google.com/'

urls = []    # empty list for image urls to download
data = []    # empty list for storing data

In [10]:
search_item = ["Fruits", "Cars", "Machine Learning","Guitar","Cakes"]

In [12]:
for item in search_item:
    driver.get(url)  # performing a Google search
    time.sleep(3)
    search_bar = driver.find_element(By.XPATH,"/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input")     # finding search bar on Google page   
    search_bar.send_keys(str(item))      # send search item list    
    search_button =driver.find_element(By.XPATH,'//span[@class="z1asCe MZy1Rb"]').click() # now we will click on search button
    
    # scroll the page in order to generate more images on the website
    for _ in range(500):
        driver.execute_script("window.scrollBy(0,100)")        
        imgs = driver.find_elements(By.XPATH,"//img[@class='rg_i Q4LuWd']")
    img_url = []
    for image in imgs:
        source = image.get_attribute('src')
        if source is not None:
                if(source[0:4] == 'http'):
                    img_url.append(source)
    for i in img_url[:10]:      # getting only 10 images each
        urls.append(i)
                    
for i in range(len(urls)):
    if i >= 500:
        break
    print("Obtaining {0} of {1} images".format(i, 100))
    response=requests.get(urls[i])
    file = open(r"C:\Users\megha\FLIPROBO\GOOGLE PICKr"+str(i)+".jpg", "wb")
    file.write(response.content)

Obtaining 0 of 100 images
Obtaining 1 of 100 images
Obtaining 2 of 100 images
Obtaining 3 of 100 images
Obtaining 4 of 100 images
Obtaining 5 of 100 images
Obtaining 6 of 100 images
Obtaining 7 of 100 images
Obtaining 8 of 100 images
Obtaining 9 of 100 images
Obtaining 10 of 100 images
Obtaining 11 of 100 images
Obtaining 12 of 100 images
Obtaining 13 of 100 images
Obtaining 14 of 100 images
Obtaining 15 of 100 images
Obtaining 16 of 100 images
Obtaining 17 of 100 images
Obtaining 18 of 100 images
Obtaining 19 of 100 images
Obtaining 20 of 100 images
Obtaining 21 of 100 images
Obtaining 22 of 100 images
Obtaining 23 of 100 images
Obtaining 24 of 100 images
Obtaining 25 of 100 images
Obtaining 26 of 100 images
Obtaining 27 of 100 images
Obtaining 28 of 100 images
Obtaining 29 of 100 images
Obtaining 30 of 100 images
Obtaining 31 of 100 images
Obtaining 32 of 100 images
Obtaining 33 of 100 images
Obtaining 34 of 100 images
Obtaining 35 of 100 images
Obtaining 36 of 100 images
Obtaining 3

In [13]:
driver.close

<bound method WebDriver.close of <selenium.webdriver.chrome.webdriver.WebDriver (session="b48400f98d30e307da76966b94cde582")>>

# Q4. Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com and scrape following details for all the search results displayed on 1st page. Details to be scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, “Secondary Camera”, “Display Size”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the details is missing then replace it by “- “. Save your results in a dataframe and CSV.

In [14]:
# connect to wed browser
driver = webdriver.Chrome(r"chromedriver.exe")

# To get the web page 
url = "https://www.flipkart.com/"
driver.get(url)
time.sleep(3)
# to close login pop up
driver.find_element(By.XPATH,"//button[@class='_2KpZ6l _2doB4z']").click()
# to enter the item name which user wants to search
item = input(" Enter the name of Smartphone that has to be searched : ")

#giving input key word to search bar
serch_bar = driver.find_element(By.XPATH,"//div[@class='_3OO5Xc']//input")
serch_bar.send_keys(item)

# to click search button
srch_btn = driver.find_element(By.XPATH,"/html/body/div[1]/div/div[1]/div[1]/div[2]/div[2]/form/div/button")
srch_btn.click()
time.sleep(5)

 Enter the name of Smartphone that has to be searched : Oneplus Nord


In [15]:
# To get  urls of phones on 1st page
page1_urls = []
urls = driver.find_elements(By.XPATH,'//a[@class="_1fQZEK"]')
for url in urls:
    page1_urls.append(url.get_attribute("href"))
len(page1_urls)

24

In [16]:
# to make emoty list for all features to be scraped
Smartphones = {}
Smartphones["Brand"] = []
Smartphones["Phone name"] = []
Smartphones["Colour"] = []
Smartphones["RAM"] = []
Smartphones["Storage(ROM)"] = []
Smartphones["Primary Camera"] = []
Smartphones["Secondary Camera"] = []
Smartphones["Display Size"] = []
Smartphones["Battery Capacity"] = []
Smartphones["Price"] = []
Smartphones["URL"] = []
# Scraping data from each url of page 1
for url in page1_urls:
    driver.get(url)                                                        
    print("Scraping URL = ", url)
    Smartphones['URL'].append(url)                                                          
    time.sleep(2)
    
    
    #Clicking on read more button
    try:
        read_more = driver.find_element(By.XPATH,'//button[@class="_2KpZ6l _1FH0tX"]')     
        read_more.click()
    except NoSuchElementException:
        print("Exception occured while moving to next page")
    
    
    #Scraping brand name of phone data
    try:
        brand_tags = driver.find_element(By.XPATH,'//span[@class="B_NuCI"]')      
        Smartphones["Brand"].append(brand_tags.text.split()[0])
    except NoSuchElementException:
        Smartphones['Brand'].append('-')
    
    
    #Scraping phone name data
    try:
        name_tags = driver.find_element(By.XPATH,'//li[@class="_21lJbe"]')     
        Smartphones['Phone name'].append(name_tags.text)
    except NoSuchElementException:
        Smartphones['Phone name'].append('-')
    
    
    #Scraping phone color data
    try:
        color_tags = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][1]/table/tbody/tr[4]/td[2]/ul/li')      
        Smartphones['Colour'].append(color_tags.text)
    except NoSuchElementException:
        Smartphones['Colour'].append('-')
        
      #Scraping RAM data
    try:
        ram_tags = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][4]/table[1]/tbody/tr[2]/td[2]/ul/li')                
        Smartphones['RAM'].append(ram_tags.text)
    except NoSuchElementException:
        Smartphones['RAM'].append('-')
    
    
    #Scraping ROM data
    try:
        rom_tags = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][4]/table[1]/tbody/tr[1]/td[2]/ul/li')        
        Smartphones['Storage(ROM)'].append(rom_tags.text)
    except NoSuchElementException:
        Smartphones['Storage(ROM)'].append('-')
        
        
    #Scraping Primary camera data
    try:                                                                                    
        pri_tags = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[2]/td[2]/ul/li')
        Smartphones['Primary Camera'].append(pri_tags.text)
    except NoSuchElementException:
        Smartphones['Primary Camera'].append('-')
        
        
    #Scraping secondary camera data
    try:                                                                                    
        sec_tags = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[6]/td[1]')
        if sec_tags != "Secondary Camera" : 
            if driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[5]/td[1]').text == "Secondary Camera":
                sec_cam = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[5]/td[2]/ul/li')
            else :
                raise NoSuchElementException
        else :
            sec_cam = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[6]/td[2]/ul/li')
        Smartphones['Secondary Camera'].append(sec_cam.text)
    except NoSuchElementException:
        Smartphones['Secondary Camera'].append('-')
        
    
    #Scraping Display size data 
    try:
        disp_tags = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][2]/div')
        if disp_tags.text != "Display Features" : raise NoSuchElementException
        disp_size = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][2]/table[1]/tbody/tr[1]/td[2]/ul/li')  
        Smartphones['Display Size'].append(disp_size.text)
    except NoSuchElementException:
        Smartphones['Display Size'].append('-')
     

    
    #Scraping battery capacity data
    try:
        if driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][10]/div').text != "Battery & Power Features" :
            if driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][9]/div').text == "Battery & Power Features" :
                bat_tags = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][9]/table/tbody/tr/td[1]')
                if bat_tags.text != "Battery Capacity" : raise NoSuchElementException
                bat_capa = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][9]/table/tbody/tr/td[2]/ul/li')                
            elif driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][8]/div').text == "Battery & Power Features" :
                bat_tags = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][8]/table/tbody/tr/td[1]')
                if bat_tags.text != "Battery Capacity" : raise NoSuchElementException
                bat_capa = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][8]/table/tbody/tr/td[2]/ul/li')
            else:
                raise NoSuchElementException
        else :
            bat_tags = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][10]/table/tbody/tr/td[1]')
            if bat_tags.text != "Battery Capacity" : raise NoSuchElementException
            bat_capa = driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][10]/table/tbody/tr/td[2]/ul/li')              
        Smartphones['Battery Capacity'].append(bat_capa.text)
    except NoSuchElementException:
        Smartphones['Battery Capacity'].append('-')
        
     #Scraping Price data
    try:
        price_tags = driver.find_element(By.XPATH,'//div[@class="_30jeq3 _16Jk6d"]')      
        Smartphones['Price'].append(price_tags.text)
    except NoSuchElementException:
        Smartphones['Price'].append('-')

Scraping URL =  https://www.flipkart.com/oneplus-nord-ce-2-lite-5g-black-dusk-128-gb/p/itm7acae55b999e6?pid=MOBGHH9FVUHCFHTY&lid=LSTMOBGHH9FVUHCFHTYTOIZQW&marketplace=FLIPKART&q=Oneplus+Nord&store=tyy%2F4io&srno=s_1_1&otracker=search&otracker1=search&fm=organic&iid=74ffcfd9-0420-41aa-bfa3-9091d0631d4c.MOBGHH9FVUHCFHTY.SEARCH&ppt=hp&ppn=homepage&ssid=10i9du3odc0000001666951634500&qH=53863d4d978d5074
Scraping URL =  https://www.flipkart.com/oneplus-nord-ce-5g-charcoal-ink-128-gb/p/itmd6dcbd8116ed1?pid=MOBGDM4D6RNZR2SA&lid=LSTMOBGDM4D6RNZR2SA7IOH3F&marketplace=FLIPKART&q=Oneplus+Nord&store=tyy%2F4io&srno=s_1_2&otracker=search&otracker1=search&fm=organic&iid=74ffcfd9-0420-41aa-bfa3-9091d0631d4c.MOBGDM4D6RNZR2SA.SEARCH&ppt=hp&ppn=homepage&ssid=10i9du3odc0000001666951634500&qH=53863d4d978d5074
Scraping URL =  https://www.flipkart.com/oneplus-nord-gray-onyx-256-gb/p/itm49f817b591982?pid=MOBFUE5GGWHFC37J&lid=LSTMOBFUE5GGWHFC37JNXZLYZ&marketplace=FLIPKART&q=Oneplus+Nord&store=tyy%2F4io&srno=s_1

Scraping URL =  https://www.flipkart.com/oneplus-nord-2-5g-gray-sierra-256-gb/p/itmc88e4a702db46?pid=MOBGDBFRAPSY7GFZ&lid=LSTMOBGDBFRAPSY7GFZK4SRDS&marketplace=FLIPKART&q=Oneplus+Nord&store=tyy%2F4io&srno=s_1_22&otracker=search&otracker1=search&fm=organic&iid=74ffcfd9-0420-41aa-bfa3-9091d0631d4c.MOBGDBFRAPSY7GFZ.SEARCH&ppt=hp&ppn=homepage&ssid=10i9du3odc0000001666951634500&qH=53863d4d978d5074
Scraping URL =  https://www.flipkart.com/oneplus-nord-2t-5g-gray-shadow-256-gb/p/itm97eda8b1a1566?pid=MOBGFX6GFUSXWRQN&lid=LSTMOBGFX6GFUSXWRQNGOM82S&marketplace=FLIPKART&q=Oneplus+Nord&store=tyy%2F4io&srno=s_1_23&otracker=search&otracker1=search&fm=organic&iid=74ffcfd9-0420-41aa-bfa3-9091d0631d4c.MOBGFX6GFUSXWRQN.SEARCH&ppt=hp&ppn=homepage&ssid=10i9du3odc0000001666951634500&qH=53863d4d978d5074
Scraping URL =  https://www.flipkart.com/oneplus-nord-2t-5g-jade-fog-256-gb/p/itm97eda8b1a1566?pid=MOBGFX6PB4EGYBW8&lid=LSTMOBGFX6PB4EGYBW8KGGLEN&marketplace=FLIPKART&q=Oneplus+Nord&store=tyy%2F4io&srno=s_1_

In [17]:
#Checking lengths of all scraped data
print(len(Smartphones["Brand"]), len(Smartphones["Phone name"]), len(Smartphones["Colour"]), len(Smartphones["RAM"]), len(Smartphones["Storage(ROM)"]))

24 24 24 24 24


In [18]:
#Framing the data
df = pd.DataFrame.from_dict(Smartphones)
df


,Brand,Phone name,Colour,RAM,Storage(ROM),Primary Camera,Secondary Camera,Display Size,Battery Capacity,Price,URL
0,OnePlus,"Phone,SIM Tray Ejector, Adapter, Phone Case, U...",Black Dusk,6 GB,128 GB,Primary & Secondary Camera,-,16.74 cm (6.59 inch),-,"₹19,999",https://www.flipkart.com/oneplus-nord-ce-2-lit...
1,OnePlus,"Handset,Warp Charge 30T power adapter Warp Typ...",Charcoal Ink,8 GB,128 GB,Primary Camera,-,16.33 cm (6.43 inch),-,"₹24,990",https://www.flipkart.com/oneplus-nord-ce-5g-ch...
2,OnePlus,"Screen protector (pre-applied), Warp Charge 30...",Gray Onyx,12 GB,256 GB,48MP + 8MP,-,16.36 cm (6.44 inch),4115 mAh,"₹29,999",https://www.flipkart.com/oneplus-nord-gray-ony...
3,OnePlus,"Screen protector (pre-applied), Warp Charge 30...",Gray Onyx,6 GB,64 GB,48MP + 8MP,-,16.36 cm (6.44 inch),4115 mAh,"₹24,999",https://www.flipkart.com/oneplus-nord-gray-ony...
4,OnePlus,"Screen protector (pre-applied), Warp Charge 30...",Blue Marble,12 GB,256 GB,48MP + 8MP,-,16.36 cm (6.44 inch),4115 mAh,"₹29,999",https://www.flipkart.com/oneplus-nord-blue-mar...
5,OnePlus,"Screen protector (pre-applied), Warp Charge 30...",Gray Onyx,8 GB,128 GB,48MP + 8MP,-,16.36 cm (6.44 inch),4115 mAh,"₹27,999",https://www.flipkart.com/oneplus-nord-gray-ony...
6,OnePlus,"Screen protector (pre-applied), Warp Charge 30...",Blue Marble,8 GB,128 GB,48MP + 8MP,-,16.36 cm (6.44 inch),4115 mAh,"₹27,999",https://www.flipkart.com/oneplus-nord-blue-mar...
7,OnePlus,"Screen protector (pre-applied), Warp Charge 30...",Blue Marble,6 GB,64 GB,48MP + 8MP,-,16.36 cm (6.44 inch),4115 mAh,"₹24,999",https://www.flipkart.com/oneplus-nord-blue-mar...
8,OnePlus,"Phone,SIM Tray Ejector, Adapter, Phone Case, U...",Blue Tide,6 GB,128 GB,Primary & Secondary Camera,-,16.74 cm (6.59 inch),-,"₹19,999",https://www.flipkart.com/oneplus-nord-ce-2-lit...
9,OnePlus,"Handset,SIM Tray Ejector, Adapter, Phone Case,...",Black Dusk,6 GB,128 GB,Primary Camera,-,16.74 cm (6.59 inch),-,"₹19,950",https://www.flipkart.com/oneplus-nord-ce-2-lit...


In [19]:
driver.close

<bound method WebDriver.close of <selenium.webdriver.chrome.webdriver.WebDriver (session="c995e1651d13244e59faee0e47bac419")>>

# Q5.Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps.

In [20]:
# to get web browser
driver=webdriver.Chrome(r"chromedriver.exe") 
time.sleep(2)

# opening google maps
url = "https://www.google.co.in/maps"
driver.get(url)
time.sleep(2)


#Sending keyword for seach bar and search button
city = input('Enter City name that has to be searched : ')
search_bar = driver.find_element(By.ID,"searchboxinput")                       
search_bar.clear()                                                             
time.sleep(2)
search_bar.send_keys(city)                                                     
search_btn = driver.find_element(By.ID,"searchbox-searchbutton")              
search_btn.click()                                                             
time.sleep(3)

try:
    url_string = driver.current_url
    print("URL Extracted: ", url_string)
    lat_lng = re.findall(r'@(.*)data',url_string)
    if len(lat_lng):
        lat_lng_list = lat_lng[0].split(",")
        if len(lat_lng_list)>=2:
            lat = lat_lng_list[0]
            lng = lat_lng_list[1]
        print("Latitude = {}, Longitude = {}".format(lat, lng))

except Exception as e:
        print("Error: ", str(e))

Enter City name that has to be searched : PATNA
URL Extracted:  https://www.google.co.in/maps/place/Patna,+Bihar/@25.6079707,85.0029556,11z/data=!3m1!4b1!4m5!3m4!1s0x39f29937c52d4f05:0x831a0e05f607b270!8m2!3d25.5940947!4d85.1375645
Latitude = 25.6079707, Longitude = 85.0029556


In [21]:
driver.close

<bound method WebDriver.close of <selenium.webdriver.chrome.webdriver.WebDriver (session="539205edbabc387ba26a713b800563bf")>>

# Q6. Write a program to scrap details of all the funding deals for second quarter (i.e Jan 21 – March 21) from trak.in.

In [22]:
# to get web browser
driver=webdriver.Chrome(r"chromedriver.exe") 
time.sleep(2)

#to get url
driver.get('https://trak.in/')

#to get xpath for funding deals
fund_button = driver.find_element(By.XPATH,'//li[@id="menu-item-51510"]/a').get_attribute('href')
driver.get(fund_button)

In [23]:
# Empty lists
fund_dict = {}
fund_dict['Date'] = []
fund_dict['Startup Name'] = []
fund_dict['Industry/Vertical'] = []
fund_dict['Sub-Vertical'] = []
fund_dict['Location'] = []
fund_dict['Investor'] = []
fund_dict['Investment Type'] = []
fund_dict['Amount(in USD)'] = []


for i in range(54,57):
    # Date
    dt = driver.find_elements(By.XPATH,'//table[@id="tablepress-{}"]/tbody/tr/td[2]'.format(i))
    for d in dt:
        fund_dict['Date'].append(d.text)

    # Startup Name
    sn = driver.find_elements(By.XPATH,'//table[@id="tablepress-{}"]/tbody/tr/td[3]'.format(i))
    for n in sn:
        fund_dict['Startup Name'].append(n.text)
    
    # Industry/Vertical
    ind = driver.find_elements(By.XPATH,'//table[@id="tablepress-{}"]/tbody/tr/td[4]'.format(i))
    for n in ind:
        fund_dict['Industry/Vertical'].append(n.text)
    
    # Sub-Vertical
    sv = driver.find_elements(By.XPATH,'//table[@id="tablepress-{}"]/tbody/tr/td[5]'.format(i))
    for s in sv:
        fund_dict['Sub-Vertical'].append(s.text)

    # Location
    loc = driver.find_elements(By.XPATH,'//table[@id="tablepress-{}"]/tbody/tr/td[6]'.format(i))
    for l in loc:
        fund_dict['Location'].append(l.text)
    
      # Investor
    inv = driver.find_elements(By.XPATH,'//table[@id="tablepress-{}"]/tbody/tr/td[7]'.format(i))
    for n in inv:
        fund_dict['Investor'].append(n.text)
    
    # Investment Type
    invt = driver.find_elements(By.XPATH,'//table[@id="tablepress-{}"]/tbody/tr/td[8]'.format(i))
    for n in invt:
        fund_dict['Investment Type'].append(n.text)
    
    # Amount
    amt = driver.find_elements(By.XPATH,'//table[@id="tablepress-{}"]/tbody/tr/td[9]'.format(i))
    for a in amt:
        fund_dict['Amount(in USD)'].append(a.text)
    
fund_df = pd.DataFrame(fund_dict)
fund_df

,Date,Startup Name,Industry/Vertical,Sub-Vertical,Location,Investor,Investment Type,Amount(in USD)
0,15/01/2021,Digit Insurance,Financial Services,Insurance Services,Bengaluru,"A91 Partners, Faering Capital, TVS Capital Funds",Venture,"1,80,00,000"
1,28/01/2021,Bombay Shaving Company,Consumer Goods Company,"Shave care, beard care, and skincare products",New Delhi,Reckitt Benckiser,Venture,"6,172,258.50"
2,19/01/2021,DeHaat,AgriTech Startup,online marketplace for farm products and services,Patna,Prosus Ventures,Series C,"30,000,000"
3,19/01/2021,Darwinbox,SaaS,HR Tech,Mumbai,Salesforce Ventures,Seed,"15,000,000"
4,18/01/2021,mfine,Health Tech Startup,AI-powered telemedicine mobile app,Bengaluru,Heritas Capital Management,Venture Round,"16,000,000"
5,18/01/2021,Udayy,EdTech,Online learning platform for kids in class 1-5,Gurgaon,Sequoia Capital,Seed Funding,"15,000,000"
6,11/01/2021,True Elements,Food Startup,Whole Food plant based Nashta,Pune,SIDBI Venture Capital,Series,"100,000,000"
7,13/01/2021,Saveo,B2B E-commerce,Pharmacies,Bengaluru,"Matrix Partners India, RTP Global, others",Seed,"4,000,000"
8,11/02/2021,Doubtnut,Edu Tech,E-Learning Platform,Gurgaon,"SIG Global, Sequoia Capital, WaterBridge Ventu...",Series B,"2,500,000"
9,22/02/2021,Zomato,Hospitality,Online Food Delivery Platform,Gurgaon,"Tiger Global, Kora",Venture,"250,000,000"


In [24]:
driver.close

<bound method WebDriver.close of <selenium.webdriver.chrome.webdriver.WebDriver (session="f7a44199675aab27987c669c5f1118fe")>>

# Q7.Write a program to scrap all the available details of best gaming laptops from digit.in.

In [25]:
# Connecting to the webdriver
driver=webdriver.Chrome(r"chromedriver.exe")
# opening the url digit.in
url = "https://www.digit.in/"
driver.get(url)
time.sleep(2)
#searching for best laptop
best_gam_laptops = driver.find_element(By.XPATH,"//div[@class='listing_container']//ul//li[9]").click()
time.sleep(4)
#Creating empty lists
Laptop_Name = []
Operating_sys = []
Display = []
Processor = []
Memory = []
Weight = []
Dimensions = []
Graph_proc = []
Price = []

In [26]:


# Scraping the data of laptop names
laptop_name= driver.find_elements(By.XPATH,'//div[@class="TopNumbeHeading"]/div/a/h3')
for name in laptop_name:
    Laptop_Name.append(name.text)
    
    
#Scraping the data of operating system
try:
    op_sys = driver.find_elements(By.XPATH,"//div[@class='product-detail']/div/ul/li[1]/div/div")
    for os in op_sys:
        Operating_sys.append(os.text)
except NoSuchElementException:
    pass


#Scraping data of display of the laptop
try:
    display= driver.find_elements(By.XPATH,"//div[@class='product-detail']/div/ul/li[2]/div/div")
    for disp in display:
        Display.append(disp.text)
except NoSuchElementException:
    pass


#Scraping data of Processor
try:
    processor = driver.find_elements(By.XPATH,"//div[@class='Spcs-details'][1]/table/tbody/tr[5]/td[3]")
    for pro in processor:
        Processor.append(pro.text)
except NoSuchElementException:
    pass

#Scraping data of memory
try:
    memory = driver.find_elements(By.XPATH,"//div[@class='Spcs-details'][1]/table/tbody/tr[6]/td[3]")
    for memo in memory:
        Memory.append(memo.text)
except NoSuchElementException:
    pass

#Scraping data of weight
try:
    weight = driver.find_elements(By.XPATH,"//div[@class='Spcs-details'][1]/table/tbody/tr[7]/td[3]")
    for wgt in weight:
        Weight.append(wgt.text)
except NoSuchElementException:
    pass


#Scraping data of dimensions
try:
    dimension = driver.find_elements(By.XPATH,"//div[@class='Spcs-details'][1]/table/tbody/tr[8]/td[3]")
    for dim in dimension:
        Dimensions.append(dim.text)
except NoSuchElementException:
    pass


#Scraping data of Graph processor
try:
    graph = driver.find_elements(By.XPATH,'//div[@class="Spcs-details"]/table/tbody/tr[7]/td[3]')
    for gra in graph:
        Graph_proc.append(gra.text)
except NoSuchElementException:
    pass


#Scraping data of price
try:
    price = driver.find_elements(By.XPATH,"//td[@class='smprice']")
    for pri in price:
        Price.append(pri.text.replace('₹','Rs '))
except NoSuchElementException:
    pass

In [27]:
print(len(Laptop_Name),len(Operating_sys),len(Display),len(Processor),len(Memory),len(Weight),len(Dimensions),len(Graph_proc),len(Price))


10 10 10 10 10 10 10 10 10


In [28]:
data=list(zip(Laptop_Name, Operating_sys, Display, Processor, Memory, Weight, Dimensions, Graph_proc, Price))

In [29]:
df=pd.DataFrame(data,columns=['Laptop_Name','Operating_System','Display','Processor','Memory','Weight','Dimension','Graph_Proc','Price'])

In [30]:
df

,Laptop_Name,Operating_System,Display,Processor,Memory,Weight,Dimension,Graph_Proc,Price
0,MSI Titan GT77-12UHS,Windows 11 Home OS,"17.3"" (3840 x 2160) Display",Windows 11 Home,64 GB DDR5 RAM & 2 TB SSD,16 GB DDR6 NVIDIA GeForce RTX 3080 Ti Graphics...,397 x 330 x 23 mm dimension & 3.3 kg weight,16 GB DDR6 NVIDIA GeForce RTX 3080 Ti Graphics...,N/A
1,Alienware X17 R2,Windows 11 Home OS,"17.3"" (1920 x 1080) Display",Windows 11 Home,32 GB DDR5 RAM & 1 TB SSD,16 GB DDR6 NVIDIA GEFORCE RTX 3080 Ti Graphics...,299 x 359 x 21 mm dimension & 2.96 kg weight,16 GB DDR6 NVIDIA GEFORCE RTX 3080 Ti Graphics...,"Rs 389,990"
2,Acer Predator Triton 500 SE PT516-52s,Windows 11 Home OS,"16"" (2560 x 1600) Display",Windows 11 Home,32 GB DDR5 RAM & 2 TB SSD,8 GB DDR6 NVIDIA GeForce RTXTM 3070 Ti Graphic...,358 x 262 x 19.9 mm dimension & 2.4 kg weight,8 GB DDR6 NVIDIA GeForce RTXTM 3070 Ti Graphic...,"Rs 300,000"
3,Omen By HP (16-B1371TX),Windows 11 Home OS,"16.1"" (2560 x 1440) Display",Windows 11 Home,8 GB DDR5 RAM & 1 TB SSD,8 GB GDDR6 NVIDIA GeForce RTX 3070 Graphics card,369 x 248 x 23 mm dimension & 2.32 kg weight,8 GB GDDR6 NVIDIA GeForce RTX 3070 Graphics card,"Rs 176,290"
4,Acer Predator Helios 300 AN515-45 (NH.QBRSI.0,Windows 11 Home OS,"15.6"" (2560 x 1440) Display",Windows 11 Home,16 GB DDR4 RAM & 512 GB SSD,8 GB DDR6 NVIDIA GeForce RTX 3070 Graphics card,363 x 255 x 23.9 mm dimension & 2.4 kg weight,8 GB DDR6 NVIDIA GeForce RTX 3070 Graphics card,"Rs 172,999"
5,MSI Delta 15 (A5EFK-083IN),Windows 11 Home OS,"15.6"" (1920 x 1080) Display",Windows 11 Home,16 GB DDR4 RAM & 1 TB SSD,10 GB DDR6 AMD Radeon RX 6700M Graphics card,357 x 247 x 19 mm dimension & 1.9 kg weight,10 GB DDR6 AMD Radeon RX 6700M Graphics card,"Rs 188,990"
6,Omen By HP (16-C0141AX),Windows 11 Home OS,"16.1"" (2560 x 1440) Display",Windows 11 Home,16 GB DDR4 RAM & 1 TB NVMe,8 GB GDDR6 AMD Radeon™ RX 6600M Graphics card,36.92 x 24.8 x 2.3 mm dimension & 2.3 kg weight,8 GB GDDR6 AMD Radeon™ RX 6600M Graphics card,"Rs 124,990"
7,Lenovo Legion 5i Pro (82RF00MGIN),Windows 11 Home OS,"16"" (2560 x 1600) Display",Windows 11 Home,16 GB DDR5 RAM & 1 TB SSD,6 GB DDR6 NVIDIA GeForce RTX 3060 Graphics card,360 x 264 x 20 mm dimension & 2.49 kg weight,6 GB DDR6 NVIDIA GeForce RTX 3060 Graphics card,"Rs 168,500"
8,Alienware M15 R5 Ryzen Edition Icc-C780001win,Windows 11 Home OS,"15.6"" (1920 x 1080) Display",Windows 11 Home,16 GB DDR4 RAM & 512 GB SSD,6 GB DDR6 NVIDIA GeForce RTX 3060 Graphics card,356.2 x 272.5 x 22.85 mm dimension & 2.69 kg w...,6 GB DDR6 NVIDIA GeForce RTX 3060 Graphics card,"Rs 144,990"
9,Lenovo Slim 7 Gen 6 (82K8002JIN),Windows 11 Home OS,15.6 MP | NA Display,Windows 11 Home,16 GB DDR4 RAM & 1 TB SSD,6 GB DDR6 NVIDIA GeForce 3060 Max-Q Graphics card,356 x 252 x 16 mm dimension & 1.9 kg weight,6 GB DDR6 NVIDIA GeForce 3060 Max-Q Graphics card,"Rs 131,990"


# Q8.Write a python program to scrape the details for all billionaires from www.forbes.com. Details to be scrapped: “Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, “Industry”.

In [40]:
import selenium
from selenium import webdriver
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
from selenium.webdriver.common.by import By

import pandas as pd

import warnings
warnings.filterwarnings('ignore')

import time

In [41]:
driver = webdriver.Chrome(r"chromedriver.exe")
driver.get("https://www.forbes.com/billionaires/")
time.sleep(3)

In [42]:
# Scraping name of billionaires
name=[]
try:
    n = driver.find_elements(By.XPATH,'//div[@class="personName"]')
    for i in n:
        name.append(i.text)
except NoSuchElementException:
    name.append('-')

# Scraping rank of billionaires
rank=[]    
try:    
    r = driver.find_elements(By.XPATH,'//div[@class="rank"]')
    for i in r:
        rank.append(i.text)
except NoSuchElementException:
    rank.append('-')
        
        
# Scraping networth of billionaires        
nworth=[]
try:
    nw= driver.find_elements(By.XPATH,'//div[@class="netWorth"]')
    for i in nw:
        nworth.append(i.text)
except NoSuchElementException:
    nworth.append('-')

# Scraping age of billionaires    
age=[]
try:
    a= driver.find_elements(By.XPATH,'//div[@class="age"]')
    for i in a:
        age.append(i.text)
except NoSuchElementException:
    age.append('-')

# Scraping citizenship of billionaires    
citizenship=[]
try:
    c = driver.find_elements(By.XPATH,'//div[@class="countryOfCitizenship"]')
    for i in c:
        citizenship.append(i.text)
except NoSuchElementException:
    citizenship.append('-')
    
    
# Scraping source of billionaires    
source=[]
try:
    s = driver.find_elements(By.XPATH,'//div[@class="source"]')
    for i in s:
        source.append(i.text)
except NoSuchElementException:
    source.append('-')
    
# Scraping industry of billionaires
industry=[]
try:
    ind = driver.find_elements(By.XPATH,'//div[@class="category"]')
    for i in ind:
        industry.append(i.text)
except NoSuchElementException:
    industry.append('-')

In [43]:
print(len(name),len(rank),len(nworth),len(age),len(industry),len(citizenship),len(source)) #checking length of each element

200 200 200 200 200 200 200


In [44]:
Forbes = pd.DataFrame({'Rank':rank,'Name':name,'Age':age,"Net_Worth":nworth,"Citizenship":citizenship,"Source":source,"Industry":industry})
Forbes   #final Dataframe

,Rank,Name,Age,Net_Worth,Citizenship,Source,Industry
0,1.,Elon Musk,50,$219 B,United States,"Tesla, SpaceX",Automotive
1,2.,Jeff Bezos,58,$171 B,United States,Amazon,Technology
2,3.,Bernard Arnault & family,73,$158 B,France,LVMH,Fashion & Retail
3,4.,Bill Gates,66,$129 B,United States,Microsoft,Technology
4,5.,Warren Buffett,91,$118 B,United States,Berkshire Hathaway,Finance & Investments
...,...,...,...,...,...,...,...
195,192.,Marcel Herrmann Telles,72,$10.3 B,Brazil,beer,Food & Beverage
196,197.,Leon Black,70,$10 B,United States,private equity,Finance & Investments
197,197.,Joe Gebbia,40,$10 B,United States,Airbnb,Technology
198,197.,David Geffen,79,$10 B,United States,"movies, record labels",Media & Entertainment


# Q9. Write a program to extract at least 500 Comments, Comment upvote and time when comment was posted from any YouTube Video.

In [52]:
# to get web browser
driver=webdriver.Chrome(r"chromedriver.exe") 
time.sleep(2)
# to get url
url = "https://www.youtube.com/"
driver.get(url)
time.sleep(2)

In [53]:
# input the keywords that user wants to search
user_input = input('Enter the video you want to search : ')

#finding element for search bar
search_bar = driver.find_element(By.XPATH,'//input[@id="search"]')
search_bar.send_keys(user_input)
time.sleep(1)

Enter the video you want to search : RRR


In [54]:
#clicking on search button
search_btn = driver.find_element(By.ID,"search-icon-legacy")  
search_btn.click()
time.sleep(1)

In [55]:
#clicking on first video
link_click = driver.find_element(By.XPATH,"//yt-formatted-string[@class ='style-scope ytd-video-renderer']")
link_click.click()

# 10000 time we scroll down to generate more Comments
for _ in range(10000):
    driver.execute_script("window.scrollBy(0,10000)")

In [56]:
# empty lists
comments = []
comment_time = []
Time = []
Likes = []
No_of_Likes = []

# to scrape comments
cm_tags = driver.find_elements(By.ID,"content-text")
for cm in cm_tags:
    if cm.text is None:
        comments.append("--")
    else:
        comments.append(cm.text)
time.sleep(5)
comments[:500]

['Theatrical experience of this song is unbelievable. The choreography is mind blowing.',
 'The choreography here is just fabulous and the energy showed by the actors is really appreciable amd incomparable',
 'This song is very popular in international level too. 🇮🇳\nPower of sir S. S RAJAMAULI ',
 "I cant believe it hasn't been selected in the Oscars, this film is a masterpiece ",
 'I love the energy they put in this dances.',
 "Never seen so many people from other countries appreciating an Indian song... It's truly a great song",
 "You can't sit idle while listening to this song.. marvellous energy and rhythm..a mind blowing movie.. atmosphere inside the movie hall was unbelievable...Jr NTR and Ram Charan were superb..full paisa wasool movie.. enjoyed thoroughly..Thank you Rajamouli Sir for this fantastic movie..",
 "if this movie was 100 hours long, I'd still watch it on repeat. this is exactly the message the world needs to embody with the current state of the world",
 'My sister l

In [57]:
len(comments)

4100

In [58]:
# to scrape time when comment was posted
tm_tags = driver.find_elements(By.XPATH,"//a[contains(text(),'ago')]")
for tm in tm_tags:
    if tm.text is None:
        Time.append("--")
    else:
        Time.append(tm.text)

for i in range(0,len(Time),2):
    comment_time.append(Time[i])
time.sleep(5)
comment_time[:500]

# to scrape comment likes
like_tags = driver.find_elements(By.XPATH,"//span[@class='style-scope ytd-comment-action-buttons-renderer']")
for like in like_tags:
    if like.text is None:
        Likes.append("--")
    else:
        Likes.append(like.text)
    
for i in range(1,len(Likes),2):
    No_of_Likes.append(Likes[i])
Likes[:500]    

['',
 '8.7K',
 '',
 '330',
 '',
 '22',
 '',
 '488',
 '',
 '621',
 '',
 '269',
 '',
 '351',
 '',
 '750',
 '',
 '313',
 '',
 '29',
 '',
 '342',
 '',
 '248',
 '',
 '416',
 '',
 '428',
 '',
 '29',
 '',
 '156',
 '',
 '1.3K',
 '',
 '110',
 '',
 '511',
 '',
 '341',
 '',
 '69',
 '',
 '334',
 '',
 '188',
 '',
 '61',
 '',
 '303',
 '',
 '691',
 '',
 '753',
 '',
 '61',
 '',
 '1K',
 '',
 '10',
 '',
 '3.4K',
 '',
 '137',
 '',
 '746',
 '',
 '150',
 '',
 '143',
 '',
 '45',
 '',
 '1.7K',
 '',
 '112',
 '',
 '14',
 '',
 '89',
 '',
 '55',
 '',
 '14',
 '',
 '212',
 '',
 '155',
 '',
 '144',
 '',
 '56',
 '',
 '25',
 '',
 '5',
 '',
 '706',
 '',
 '122',
 '',
 '130',
 '',
 '74',
 '',
 '88',
 '',
 '5',
 '',
 '28',
 '',
 '57',
 '',
 '192',
 '',
 '109',
 '',
 '6',
 '',
 '234',
 '',
 '23',
 '',
 '15',
 '',
 '98',
 '',
 '46',
 '',
 '7',
 '',
 '550',
 '',
 '83',
 '',
 '26',
 '',
 '322',
 '',
 '94',
 '',
 '5',
 '',
 '161',
 '',
 '87',
 '',
 '592',
 '',
 '63',
 '',
 '59',
 '',
 '3',
 '',
 '444',
 '',
 '82',
 '',
 '21',

In [59]:
#Creating dataframe
Youtube=pd.DataFrame({})
Youtube['Comments'] = comments[:500]
Youtube['Comment_time'] = comment_time[:500]
Youtube['Comment upvotes'] = No_of_Likes[:500]

#Printing dataframe
Youtube

,Comments,Comment_time,Comment upvotes
0,Theatrical experience of this song is unbeliev...,6 months ago,8.7K
1,The choreography here is just fabulous and the...,6 days ago,330
2,This song is very popular in international lev...,1 month ago,22
3,I cant believe it hasn't been selected in the ...,6 months ago,488
4,I love the energy they put in this dances.,1 month ago (edited),621
...,...,...,...
495,1:00 my fav part I enjoy it a lot,2 months ago,
496,Energy of this song is best experienced in the...,3 months ago,1
497,Energy level,6 months ago,1
498,Will be watching this song multiple times ever...,6 months ago,54


In [60]:
driver.close()

# Q10. Write a python program to scrape a data for all available Hostels from https://www.hostelworld.com/ in “London” location. You have to scrape hostel name, distance from city centre, ratings, total reviews, overall reviews, privates from price, dorms from price, facilities and property description.

In [ ]:
# Importing Libraries
import selenium
import pandas as pd
import time
from bs4 import BeautifulSoup

# Importing selenium webdriver 
from selenium import webdriver
import warnings
warnings.filterwarnings('ignore')

# Importing required Exceptions which needs to handled
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
from selenium.webdriver.common.by import By
#Importing requests
import requests

# importing regex
import re

In [99]:
# to get web browser
driver=webdriver.Chrome(r"chromedriver.exe") 
time.sleep(3)

# to get url
url = "https://www.hostelworld.com/"
driver.get(url)

In [102]:
#to scrape location search bar
search_loc = driver.find_element(By.XPATH,'//input[@class="search-input"]')
search_loc.send_keys("London")
time.sleep(2)

In [103]:
#select london
london = driver.find_element(By.XPATH,'//div[@class="label"]')
london.click()
time.sleep(2)

In [104]:
# do click in search button
search_btn = driver.find_element(By.XPATH,'//i[@class="core-icon icon-core-search-fill"]')
search_btn.click()

In [105]:
#empty lists
hostel_name = []
distance = []
pvt_prices = []
dorms_price = []
rating = []
reviews = []
over_all = []
facilities = []
description =[]
product_url = []

In [106]:
for i in driver.find_elements(By.XPATH,"//div[@class = 'pagination-item pagination-current']"):
    i.click()
    time.sleep(4)
    #to get hostel name
    try:
        name = driver.find_elements(By.XPATH,"//h2[@class='title title-6']")
        for i in name:
            hostel_name.append(i.text)
    except NoSuchElementException:
        hostel_name.append('-')
    #to get distance from city centre
    
    try:
        dist = driver.find_elements(By.XPATH,"//div[@class='subtitle body-3']//a//span[1]")
        for i in dist:
            distance.append(i.text.replace('Hostel - ',''))
    except NoSuchElementException:
        distance.append('-')
        
    for i in driver.find_elements(By.XPATH,"//div[@class='prices-col']"):
    #to get privates from price
        try:
            pvt_price = driver.find_element(By.XPATH,"//a[@class='prices']//div[1]//div")
            pvt_prices.append(pvt_price.text)
        except NoSuchElementException:
            pvt_prices.append('-')
    #to get  dorms from price
    for i in driver.find_elements(By.XPATH,"//div[@class='prices-col']"):
        try:
            dorms = driver.find_element(By.XPATH,"//a[@class='prices']//div[2]//div")
            dorms_price.append(dorms.text)
        except NoSuchElementException:
            dorms_price.append('-')
            #to get facilities
    try:
        fac1 = driver.find_elements(By.XPATH,"//div[@class='has-wifi']")
        fac2 = driver.find_elements(By.XPATH,"//div[@class='has-sanitation']")
        for i in fac1:
            for j in fac2:
                facilities.append(i.text +', '+ j.text )
    except NoSuchElementException:
        facilities.append('-')
    #to fetch url of each hostel
    p_url = driver.find_elements(By.XPATH,"//div[@class='prices-col']//a[2]")
    for i in p_url:
        product_url.append(i.get_attribute('href'))

for i in product_url:
    driver.get(i)
    time.sleep(3)
    #to click on show more button for description
    try:
        driver.find_element(By.XPATH,'//button[@class="button primary small full-width"]').click()
        time.sleep(5)
    except NoSuchElementException:
        pass
    #to get ratings
    try:
        rat = driver.find_element(By.XPATH,"//div[@class='score orange big']")
        rating.append(rat.text)
    except NoSuchElementException:
        rating.append('-')
    # to get total reviews
        
    try:
        rws = driver.find_element(By.XPATH,"//div[@class='reviews']")
        reviews.append(rws.text.replace('Total Reviews',''))
    except NoSuchElementException:
        reviews.append('-')
        #to get  overall review
    try:
        overall_rw = driver.find_element(By.XPATH,"//div[@class='keyword']//span")
        over_all.append(overall_rw.text)
    except NoSuchElementException:
        over_all.append('-')
    #to get property description 
    try:
        disc = driver.find_element(By.XPATH,"//div[@class='content']")
        description.append(disc.text)
    except NoSuchElementException:
        over_all.append('-')

In [107]:
#to make dataframe
dff = pd.DataFrame({})
dff['Hostel_Name'] = hostel_name
dff['Distance fron city centre'] = distance
dff['Ratings'] = rating
dff['Total_reviews'] = reviews
dff['Overall Reviews'] = over_all
dff['Privates from price'] = pvt_prices
dff['Dorms from price'] = dorms_price
dff['Facilities'] = facilities[:60]
dff['Description'] = description

In [108]:
#Printing data frame
dff

,Hostel_Name,Distance fron city centre,Ratings,Total_reviews,Overall Reviews,Privates from price,Dorms from price,Facilities,Description
